In [11]:
import ccxt.pro as ccxtpro
import asyncio
import nest_asyncio
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import statsmodels.api as sm

nest_asyncio.apply()

with open('binance_api.txt', 'r') as file:
    api_key = file.readline().strip()
    secret = file.readline().strip()

exchange = ccxtpro.binance({
    'apiKey': api_key,
    'secret': secret,
    'options': {'defaultType': 'future'}  # Perpetual 선물 시장을 선택합니다.
})

In [71]:
async def fetch_symbol_data(exchange, symbol, since):
    """
    특정 심볼의 OHLCV 데이터를 가져와 일 단위로 거래량을 USDT로 변환한 후 3년치 총 거래량을 반환합니다.

    입력값:
    - exchange: ccxtpro 거래소 객체
    - symbol: 심볼 문자열
    - since: 데이터 시작 시점 (timestamp)

    출력값:
    - (symbol, total_usdt_volume, df): 심볼, 총 USDT 거래량, 데이터프레임의 튜플
    """
    try:
        ohlcv = await exchange.fetch_ohlcv(symbol, '1d', since=since)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['quote_volume'] = df['volume'] * df['close']  # 일 단위로 거래량을 USDT로 환산
        total_usdt_volume = df['quote_volume'].sum()
        return symbol, total_usdt_volume, df
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return symbol, 0, None

async def get_top_symbols_by_volume(exchange, n=40):
    """
    지난 3년 동안의 거래량을 USDT 기준으로 상위 n개의 심볼을 선택하고 해당 데이터를 반환합니다.

    입력값:
    - exchange: ccxtpro 거래소 객체
    - n: 선택할 심볼의 개수

    출력값:
    - top_symbols: (symbol, total_usdt_volume, df) 튜플의 리스트 (상위 n개 심볼의 데이터)
    """
    await exchange.load_markets()
    symbols = []

    for symbol in exchange.markets:
        if exchange.markets[symbol]['active'] and '/USDT' in symbol:
            symbols.append(symbol)

    three_years_ago = exchange.parse8601((datetime.now() - timedelta(days=5*365)).strftime('%Y-%m-%dT%H:%M:%SZ'))

    tasks = [fetch_symbol_data(exchange, symbol, three_years_ago) for symbol in symbols]
    volume_data = await asyncio.gather(*tasks)

    # 중복 제거를 위한 집합
    seen_symbols = set()
    unique_volume_data = []
    for symbol, total_usdt_volume, df in volume_data:
        clean_symbol = symbol.split(':')[0]
        perp_symbol = clean_symbol + ':USDT'
        if perp_symbol not in seen_symbols:
            seen_symbols.add(perp_symbol)
            unique_volume_data.append((symbol, total_usdt_volume, df))

    # 상위 n개의 종목 선택
    top_symbols_by_volume = sorted(unique_volume_data, key=lambda x: x[1], reverse=True)

    # 디버깅 출력
    print("Top symbols by volume before filtering:", [symbol for symbol, _, _ in top_symbols_by_volume[:50]])

    # BTC/USDT의 데이터 길이 구하기
    btc_data = [df for symbol, _, df in top_symbols_by_volume if 'BTC/USDT' in symbol]
    if not btc_data:
        raise Exception("BTC/USDT data not found in the top symbols")
    btc_length = len(btc_data[0])

    # 데이터 길이가 부족한 티커 제거 및 차순위 티커 추가
    filtered_symbols = []
    i = len(top_symbols_by_volume)
    while len(filtered_symbols) < n and top_symbols_by_volume:
        symbol, total_usdt_volume, df = top_symbols_by_volume.pop(0)
        if len(df) >= btc_length:
            filtered_symbols.append((symbol, total_usdt_volume, df))
        else:
            if i < len(unique_volume_data):
                next_symbol = unique_volume_data[i]
                i += 1
                top_symbols_by_volume.append(next_symbol)
    
    # 최종 상위 n개의 종목 선택 후 중복 제거
    perpetual_symbols = []
    seen_symbols.clear()
    for symbol, total_usdt_volume, df in filtered_symbols:
        clean_symbol = symbol.split(':')[0]
        perp_symbol = clean_symbol + ':USDT'
        if perp_symbol in exchange.markets and perp_symbol not in seen_symbols:
            perp_data = await fetch_symbol_data(exchange, perp_symbol, three_years_ago)
            if perp_data[2] is not None and len(perp_data[2]) >= btc_length:
                seen_symbols.add(perp_symbol)
                perpetual_symbols.append(perp_data)

    # 시계열적 길이 확인
    lengths = [len(df) for _, _, df in perpetual_symbols]
    print("Data lengths of selected symbols:", lengths)

    return perpetual_symbols


In [72]:
# 심볼 데이터 준비
top_symbols_data = asyncio.run(get_top_symbols_by_volume(exchange, 40))
symbols = [symbol for symbol, total_volume, df in top_symbols_data]
print(f"Top 40 symbols by volume: {symbols}")

Error fetching data for TUSD/USDT: binance does not have market symbol TUSD/USDT
Error fetching data for NULS/USDT: binance does not have market symbol NULS/USDT
Error fetching data for TFUEL/USDT: binance does not have market symbol TFUEL/USDT
Error fetching data for WIN/USDT: binance does not have market symbol WIN/USDT
Error fetching data for COS/USDT: binance does not have market symbol COS/USDT
Error fetching data for DOCK/USDT: binance does not have market symbol DOCK/USDT
Error fetching data for WAN/USDT: binance does not have market symbol WAN/USDT
Error fetching data for FUN/USDT: binance does not have market symbol FUN/USDT
Error fetching data for CTXC/USDT: binance does not have market symbol CTXC/USDT
Error fetching data for TROY/USDT: binance does not have market symbol TROY/USDT
Error fetching data for VITE/USDT: binance does not have market symbol VITE/USDT
Error fetching data for EUR/USDT: binance does not have market symbol EUR/USDT
Error fetching data for WRX/USDT: bi

In [114]:
def convert_to_utc(df):
    """
    timestamp 열을 UTC로 변환합니다.

    입력값:
    - df: OHLCV 데이터프레임

    출력값:
    - df: timestamp 열이 UTC로 변환된 데이터프레임
    """
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
    return df

def calculate_returns(df):
    """
    데이터프레임에 수익률과 누적 수익률을 계산하여 추가합니다.

    입력값:
    - df: OHLCV 데이터프레임

    출력값:
    - df: 수익률과 누적 수익률이 추가된 데이터프레임
    """
    df['returns'] = df['close'].pct_change()
    df['cumulative_returns'] = (1 + df['returns']).cumprod() - 1
    return df
    
def resample_to_monthly(df):
    """
    데이터프레임을 월별로 리샘플링하고 월별 수익률을 계산합니다.

    입력값:
    - df: OHLCV 데이터프레임

    출력값:
    - df_monthly: 월별로 리샘플링된 데이터프레임
    """
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('date', inplace=True)
    df_monthly = df.resample('M').last()
    df_monthly['returns'] = df_monthly['close'].pct_change()
    return df_monthly.reset_index()

def calculate_exponential_weights(n, decay_rate=0.3):
    """
    지수 가중치를 계산합니다.
    
    입력값:
    - n: 기간
    - decay_rate: 감쇠율 (기본값: 0.3)
    
    출력값:
    - weights: 지수 가중치 배열
    """
    weights = np.array([(1 - decay_rate) ** i for i in range(n)])
    return weights / weights.sum()

def calculate_turnover_weights(df, n=30):
    """
    일 단위 데이터에서 turnover 가중치를 계산합니다.
    
    입력값:
    - df: 일 단위 OHLCV 데이터프레임
    - n: 기간 (기본값: 30)
    
    출력값:
    - turnover_values: 가중치를 적용한 turnover 값이 포함된 데이터프레임
    """
    df = df.copy()
    df['turnover'] = df['quote_volume'] / df['quote_volume'].rolling(window=n).sum()
    df['weighted_price'] = 0.0

    weights = calculate_exponential_weights(n)
    
    for i in range(1, n + 1):
        weight = weights[i-1]
        df['weighted_price'] += weight * df['close'].shift(i)
    
    df['potential_return'] = df['close'] / df['weighted_price'] - 1
    return df[['timestamp', 'potential_return']]
    
def calculate_tk(df, window=150):
    """
    TK 값을 계산합니다.
    
    입력값:
    - df: 일 단위 OHLCV 데이터프레임
    - window: 과거 수익률을 평가할 기간 (기본값: 150일, 약 5개월)

    출력값:
    - tk_values: TK 값 리스트
    """
    df = df.copy()
    df['daily_returns'] = df['close'].pct_change()

    # Prospect Theory value 계산
    def prospect_value(r):
        return np.where(r >= 0, r ** 0.88, -2.25 * (-r) ** 0.88)

    def weight_function(n, i, total):
        return (n - i + 1) / total

    def calculate_tk_value(past_returns):
        past_returns_sorted = np.sort(past_returns)
        prospect_values = prospect_value(past_returns_sorted)
        weights = np.array([weight_function(window, i, window) for i in range(1, window + 1)])
        tk_value = np.dot(prospect_values, weights)
        return tk_value

    tk_values = [np.nan] * window
    for idx in range(window, len(df)):
        past_returns = df['daily_returns'].iloc[idx - window:idx]
        tk_value = calculate_tk_value(past_returns)
        tk_values.append(tk_value)
    
    df['tk_value'] = tk_values
    return df[['timestamp', 'tk_value']]

def calculate_overhang_values(df, window=150):
    """
    Overhang 값을 계산합니다.
    
    입력값:
    - df: 일 단위 OHLCV 데이터프레임
    - window: 과거 수익률을 평가할 기간 (기본값: 150일, 약 5개월)

    출력값:
    - df: Overhang 값 및 잠정 수익률이 포함된 데이터프레임
    """
    df = df.copy()
    df['quote_volume'] = df['volume'] * df['close']
    
    # Turnover 계산 (quote_volume을 이용)
    df['turnover'] = df['quote_volume'] / df['quote_volume'].rolling(window=window).sum()

    # 가중 평균 Turnover 계산
    weights = np.array([0.5 ** (i / window) for i in range(window)])
    weights = weights / weights.sum()
    df['weighted_turnover'] = df['turnover'].rolling(window=window).apply(lambda x: np.dot(x, weights), raw=True)

    # 가중 평균 Turnover를 이용하여 가중 가격 계산
    df['weighted_price'] = df['weighted_turnover'] * df['close']

    # 현재 잠정 수익률 계산
    df['potential_return'] = df['close'] / df['weighted_price'] - 1

    return df[['timestamp', 'potential_return']]


In [136]:
def basic_momentum_strategy(df_monthly, lookback_period=12, holding_period=1, reversal_period=1, long_threshold=0.1, short_threshold=0.1):
    """
    기본 모멘텀 전략을 적용하여 포지션을 결정하고 수익률을 계산합니다.

    입력값:
    - df_monthly: 월별로 리샘플링된 데이터프레임
    - lookback_period: 모멘텀 계산을 위한 조회 기간 (기본값: 12개월)
    - holding_period: 포지션을 유지할 기간 (기본값: 1개월)
    - reversal_period: short-term reversal을 고려한 기간 (기본값: 1개월)
    - long_threshold: 롱 포지션 청산 기준 백분위 (기본값: 상위 10%)
    - short_threshold: 숏 포지션 청산 기준 백분위 (기본값: 하위 10%)

    출력값:
    - df: 기본 모멘텀 전략의 수익률이 추가된 데이터프레임
    """
    # 모멘텀 계산 (12개월 전부터 1개월 전까지의 수익률)
    df_monthly['momentum'] = df_monthly['close'].pct_change(lookback_period).shift(-reversal_period)

    # 상위 10%와 하위 10% 임계값 계산
    long_cutoff = df_monthly['momentum'].quantile(1 - long_threshold)
    short_cutoff = df_monthly['momentum'].quantile(short_threshold)

    # 포지션 설정
    df_monthly['position'] = 0
    df_monthly.loc[df_monthly['momentum'] >= long_cutoff, 'position'] = 1
    df_monthly.loc[df_monthly['momentum'] <= short_cutoff, 'position'] = -1

    # 동일 비중 설정
    long_count = df_monthly['position'][df_monthly['position'] == 1].count()
    short_count = df_monthly['position'][df_monthly['position'] == -1].count()
    
    if long_count > 0:
        df_monthly.loc[df_monthly['position'] == 1, 'position'] = 1 / long_count
    if short_count > 0:
        df_monthly.loc[df_monthly['position'] == -1, 'position'] = -1 / short_count

    # 전략 수익률 계산
    df_monthly['basic_momentum_strategy_returns'] = df_monthly['position'].shift(1) * df_monthly['returns']
    
    return df_monthly
    
def time_series_momentum(df, lookback_period=12, long_threshold=0.1, short_threshold=0.1):
    """
    Time-Series Momentum 전략을 적용하여 포지션을 결정하고 수익률을 계산합니다.
    논문: "Time Series Momentum" by Moskowitz, Ooi, and Pedersen

    입력값:
    - df: OHLCV 데이터프레임
    - lookback_period: 모멘텀을 계산할 조회 기간 (기본값: 12개월)
    - long_threshold: 롱 포지션 설정 기준 백분위 (기본값: 상위 10%)
    - short_threshold: 숏 포지션 설정 기준 백분위 (기본값: 하위 10%)

    출력값:
    - df: Time-Series Momentum 전략의 수익률이 추가된 데이터프레임
    """
    # 모멘텀 수치 계산
    df['ts_momentum'] = df['close'].pct_change(periods=lookback_period)
    
    # 상위 10% 롱, 하위 10% 숏 포지션 설정
    df['rank'] = df['ts_momentum'].rank(pct=True)
    df['position'] = 0
    df.loc[df['rank'] > 1 - long_threshold, 'position'] = 1 / (len(df[df['rank'] > 1 - long_threshold]))
    df.loc[df['rank'] < short_threshold, 'position'] = -1 / (len(df[df['rank'] < short_threshold]))
    
    # 포지션 유지 기간 동안 수익률 계산
    df['ts_strategy_returns'] = df['position'].shift(1) * df['returns']
    
    return df

def residual_momentum(df, factors_df, lookback_period=36):
    """
    얘는 안씁니다. 36개월 회귀쳐야 하는데 애초에 데이터 수가 빡빡하고, 회귀용 학습데이터까지 끌어오느라 다른 애들이랑 동일선상에 놓기 힘듬
    이걸 동일선상에 놓으려면 최소한 전체데이터 10년치는 찍어야 합니다.
    Residual Momentum 전략을 적용하여 포지션을 결정하고 수익률을 계산합니다.
    논문: "Residual Momentum" by Blitz, Huij, and Martens

    입력값:
    - df: OHLCV 데이터프레임
    - factors_df: 팩터 데이터프레임
    - lookback_period: 잔차를 계산할 조회 기간 (기본값: 36개월)

    출력값:
    - df: Residual Momentum 전략의 수익률이 추가된 데이터프레임
    """
    df = df.copy()
    # NaN 값 제거
    df = df.dropna(subset=['returns'])
    factors_df = factors_df.dropna()

    factors = sm.add_constant(factors_df)
    model = sm.OLS(df['returns'], factors).fit()
    df['residuals'] = model.resid
    df['residual_momentum'] = df['residuals'].rolling(window=lookback_period).mean()
    df['position'] = df['residual_momentum'].apply(lambda x: 1 if x > 0 else -1)
    df['residual_strategy_returns'] = df['position'].shift(1) * df['returns']
    return df

def fifty_two_week_high(df):
    """
    52주 고가 전략을 적용하여 포지션을 결정하고 수익률을 계산합니다.
    논문: "The 52-Week High and Momentum Investing" by George and Hwang

    입력값:
    - df: 월간 리샘플링된 OHLCV 데이터프레임

    출력값:
    - df: 52주 고가 전략의 수익률이 추가된 데이터프레임
    """
    df['52_week_high'] = df['close'].rolling(window=12).max()
    df['ratio_to_high'] = df['close'] / df['52_week_high']
    df['rank'] = df['ratio_to_high'].rank(method='first', ascending=False)
    
    n = len(df)
    long_threshold = int(n * 0.1)  # 상위 10%
    short_threshold = int(n * 0.9)  # 하위 10%
    
    df['position'] = 0
    df.loc[df['rank'] <= long_threshold, 'position'] = 1
    df.loc[df['rank'] >= short_threshold, 'position'] = -1
    
    long_count = (df['rank'] <= long_threshold).sum()
    short_count = (df['rank'] >= short_threshold).sum()
    
    df['position'] = df.apply(
        lambda row: row['position'] / long_count if row['position'] == 1 else row['position'] / short_count if row['position'] == -1 else 0, 
        axis=1
    )
    
    df['52w_high_strategy_returns'] = df['position'].shift(1) * df['returns']
    return df


def capital_gains_overhang(df, potential_returns):
    """
    Capital Gains Overhang 전략을 적용하여 포지션을 결정하고 수익률을 계산합니다.
    논문: "Capital Gains Overhang and Asset Prices" by Grinblatt and Han

    입력값:
    - df: 월 단위 리샘플링된 데이터프레임
    - potential_returns: 잠정 수익률이 포함된 일 단위 데이터프레임

    출력값:
    - df: Capital Gains Overhang 전략의 수익률이 추가된 데이터프레임
    """
    df = df.copy()
    
    # 'timestamp' 열을 datetime 형식으로 변환
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
    potential_returns['timestamp'] = pd.to_datetime(potential_returns['timestamp'], unit='ms', utc=True)
    
    # 월 단위 데이터와 일 단위 잠정 수익률 병합
    df = df.merge(potential_returns, on='timestamp', how='left')
    
    # 포지션 설정
    df['rank'] = df['potential_return'].rank(method='first')
    long_threshold = df['rank'].quantile(0.9)
    short_threshold = df['rank'].quantile(0.1)

    long_count = df[df['rank'] >= long_threshold].shape[0]
    short_count = df[df['rank'] <= short_threshold].shape[0]

    df['position'] = 0
    df.loc[df['rank'] >= long_threshold, 'position'] = 1 / long_count if long_count > 0 else 0
    df.loc[df['rank'] <= short_threshold, 'position'] = -1 / short_count if short_count > 0 else 0

    df['capital_gains_strategy_returns'] = df['position'].shift(1) * df['returns']
    return df

def tk_strategy(df, tk_values, lookback_period=150):
    """
    TK 전략을 적용하여 포지션을 결정하고 수익률을 계산합니다.
    논문: "Cumulative Prospect Theory and Stock Returns" by Barberis, Huang, and Santos

    입력값:
    - df: 월 단위 OHLCV 데이터프레임
    - tk_values: 일 단위 TK 값 데이터프레임
    - lookback_period: TK 값을 계산할 조회 기간 (기본값: 150일, 약 5개월)

    출력값:
    - df: TK 전략의 수익률이 추가된 데이터프레임
    """
    df = df.copy()

    # 'timestamp' 열을 datetime 형식으로 변환
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    tk_values = convert_to_utc(tk_values)
    
    # 월 단위 데이터와 일 단위 TK 값 병합
    df = df.merge(tk_values, on='timestamp', how='left')
    df = df.ffill()  # TK 값 채우기
    
    # 포지션 설정
    df['rank'] = df['tk_value'].rank(method='first')
    long_threshold = df['rank'].quantile(0.9)
    short_threshold = df['rank'].quantile(0.1)

    long_count = df[df['rank'] >= long_threshold].shape[0]
    short_count = df[df['rank'] <= short_threshold].shape[0]

    df['position'] = 0
    df.loc[df['rank'] >= long_threshold, 'position'] = 1 / long_count if long_count > 0 else 0
    df.loc[df['rank'] <= short_threshold, 'position'] = -1 / short_count if short_count > 0 else 0

    df['tk_strategy_returns'] = df['position'].shift(1) * df['returns']
    return df


def maxing_out_strategy(df, daily_df):
    """
    Maxing Out 전략을 적용하여 포지션을 결정하고 수익률을 계산합니다.
    논문: "Maxing Out: Stocks as Lotteries and the Cross-Section of Expected Returns" by Bali, Cakici, and Whitelaw

    입력값:
    - df: 월 단위 리샘플링된 OHLCV 데이터프레임
    - daily_df: 일 단위 OHLCV 데이터프레임

    출력값:
    - df: Maxing Out 전략의 수익률이 추가된 데이터프레임
    """
    df = df.copy()

    # 'timestamp' 열을 datetime 형식으로 변환
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    daily_df = convert_to_utc(daily_df)
    
    # 일 단위 데이터에서 지난 한 달 동안의 최대 일일 수익률 계산
    daily_df['max_return'] = daily_df['close'].pct_change().rolling(window=30).max()

    # 월 단위로 리샘플링된 데이터에 일 단위 최대 수익률을 병합
    df = df.merge(daily_df[['timestamp', 'max_return']], on='timestamp', how='left')
    df.ffill(inplace=True)  # 결측치 채우기

    # 상위 10%와 하위 10%를 기반으로 포지션 설정
    df['rank'] = df['max_return'].rank(method='first')
    df['position'] = 0
    long_threshold = df['rank'].quantile(0.9)
    short_threshold = df['rank'].quantile(0.1)

    df.loc[df['rank'] >= long_threshold, 'position'] = 1
    df.loc[df['rank'] <= short_threshold, 'position'] = -1

    # 동일 가중 포트폴리오
    long_count = df[df['position'] == 1].shape[0]
    short_count = df[df['position'] == -1].shape[0]
    df['position'] = df.apply(
        lambda row: row['position'] / long_count if row['position'] == 1 else row['position'] / short_count if row['position'] == -1 else 0, 
        axis=1
    )

    # 수익률 계산
    df['maxing_out_strategy_returns'] = df['position'].shift(1) * df['returns']

    return df

def salience_strategy(df, daily_df, lookback_period=150, theta=0.88, delta=0.65):
    """
    Salience Theory 전략을 적용하여 포지션을 결정하고 수익률을 계산합니다.
    논문: "Salience Theory of Choice Under Risk" by Bordalo, Gennaioli, and Shleifer

    입력값:
    - df: 월 단위 OHLCV 데이터프레임
    - daily_df: 일 단위 OHLCV 데이터프레임
    - lookback_period: 과거 수익률을 평가할 기간 (기본값: 150일)
    - theta: Salience Theory의 theta 파라미터 (기본값: 0.88)
    - delta: Salience Theory의 delta 파라미터 (기본값: 0.65)

    출력값:
    - df: Salience Theory 전략의 수익률이 추가된 데이터프레임
    """
    def calculate_salience(returns, market_returns, theta, delta):
        salience = np.abs(returns - market_returns) ** theta / (np.abs(returns) ** theta + delta * np.abs(market_returns) ** theta)
        return salience

    df = df.copy()
    daily_df = daily_df.copy()

    # 일 단위 수익률 계산
    daily_df['daily_returns'] = daily_df['close'].pct_change()
    daily_df['market_returns'] = daily_df['close'].pct_change()

    # Salience 계산
    daily_df['salience'] = daily_df.apply(
        lambda row: calculate_salience(
            row['daily_returns'], 
            row['market_returns'], 
            theta, delta
        ),
        axis=1
    )

    # 월 단위 데이터와 일 단위 Salience 값 병합
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    daily_df = convert_to_utc(daily_df)
    df = df.merge(daily_df[['timestamp', 'salience']], on='timestamp', how='left')

    # 포지션 설정
    df['rank'] = df['salience'].rank(method='first')
    long_threshold = df['rank'].quantile(0.9)
    short_threshold = df['rank'].quantile(0.1)

    long_count = df[df['rank'] >= long_threshold].shape[0]
    short_count = df[df['rank'] <= short_threshold].shape[0]

    df['position'] = 0
    df.loc[df['rank'] >= long_threshold, 'position'] = 1 / long_count if long_count > 0 else 0
    df.loc[df['rank'] <= short_threshold, 'position'] = -1 / short_count if short_count > 0 else 0

    df['salience_strategy_returns'] = df['position'].shift(1) * df['returns']
    return df

In [128]:
def backtest_all_strategies(symbol_data, daily_data, tk_values, overhang_values, lookback_period=12, evaluation_period=1):
    """
    여러 전략을 백테스트하고 결과를 비교합니다.

    입력값:
    - symbol_data: (symbol, total_volume, df) 튜플의 리스트 (월 단위 리샘플링된 심볼 데이터)
    - daily_data: 일 단위 심볼 데이터
    - tk_values: TK 값을 포함하는 데이터프레임
    - overhang_values: Overhang 값을 포함하는 데이터프레임
    - lookback_period: 수익률 계산을 위한 조회 기간 (월 단위)
    - evaluation_period: 포트폴리오 평가 기간 (월 단위)

    출력값:
    - performance_df: 각 전략별 성과 지표 (총 수익률, MDD, Sharpe Ratio) 데이터프레임
    """
    strategies = {
        'basic_momentum_strategy': basic_momentum_strategy,
        'ts_strategy': time_series_momentum,
        '52w_high_strategy': fifty_two_week_high,
        'capital_gains_strategy': capital_gains_overhang,
        'tk_strategy': tk_strategy,
        'maxing_out_strategy': maxing_out_strategy,
        'salience_strategy': salience_strategy
    }

    all_performance = []
    
    for (symbol, total_volume, df), daily_df, tk_df, overhang_df in zip(symbol_data, daily_data, tk_values, overhang_values):
        df = calculate_returns(df)
        df = df.copy()

        for strategy_name, strategy_function in strategies.items():
            if strategy_name == 'maxing_out_strategy':
                df = strategy_function(df, daily_df)  # 일 단위 데이터도 전달
            elif strategy_name == 'tk_strategy':
                df = strategy_function(df, tk_df)  # TK 값 전달
            elif strategy_name == 'capital_gains_strategy':
                df = strategy_function(df, overhang_df)  # Overhang 값 전달
            elif strategy_name == 'salience_strategy':
                df = strategy_function(df, daily_df, lookback_period)  # 일 단위 데이터도 전달
            else:
                df = strategy_function(df)

            # 연도별로 성과 계산
            df['year'] = pd.to_datetime(df['timestamp'], unit='ms').dt.year
            for year, group in df.groupby('year'):
                performance = calculate_performance(group, strategy_name)
                performance.update({
                    'symbol': symbol,
                    'strategy': strategy_name,
                    'year': year
                })
                all_performance.append(performance)

    # 성과를 DataFrame으로 변환
    performance_df = pd.DataFrame(all_performance)
    return performance_df
    
def calculate_performance(df, strategy_name):
    """
    수익률, 최대 낙폭(MDD), Sharpe Ratio를 계산합니다.

    입력값:
    - df: OHLCV 데이터프레임
    - strategy_name: 전략 이름

    출력값:
    - performance: 성과 지표 딕셔너리 (총 수익률, MDD, Sharpe Ratio)
    """
    strategy_returns = df[f'{strategy_name}_returns']
    total_return = strategy_returns.sum()
    mdd = (strategy_returns.cumsum().cummax() - strategy_returns.cumsum()).max()
    sharpe_ratio = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)

    performance = {
        'Total Return': total_return,
        'MDD': mdd,
        'Sharpe Ratio': sharpe_ratio
    }

    return performance

In [137]:
# 심볼 데이터 준비
symbols = [symbol for symbol, total_volume, df in top_symbols_data]

# 일 단위 데이터 준비
three_years_ago = exchange.parse8601((datetime.now() - timedelta(days=5*365)).strftime('%Y-%m-%dT%H:%M:%SZ'))
daily_data = await asyncio.gather(*[fetch_symbol_data(exchange, symbol, three_years_ago) for symbol in symbols])
# 데이터프레임으로 변환
daily_data_dfs = [df for symbol, total_volume, df in daily_data if df is not None]

# TK 값 계산
tk_values = [calculate_tk(df) for df in daily_data_dfs]  # calculate_tk 함수는 TK 값을 계산하는 함수입니다.

# Overhang 값 계산
overhang_values = [calculate_overhang_values(df) for df in daily_data_dfs]  # calculate_overhang_values 함수는 Overhang 값을 계산하는 함수입니다.

# 백테스팅 실행
performance_df = backtest_all_strategies(top_symbols_data, daily_data_dfs, tk_values, overhang_values)

# 결과를 출력
performance_df

C:\Users\fawke\AppData\Local\Temp\ipykernel_35264\2555325896.py:98: RuntimeWarning: invalid value encountered in power
  return np.where(r >= 0, r ** 0.88, -2.25 * (-r) ** 0.88)
C:\Users\fawke\AppData\Local\Temp\ipykernel_35264\2555325896.py:98: RuntimeWarning: invalid value encountered in power
  return np.where(r >= 0, r ** 0.88, -2.25 * (-r) ** 0.88)
C:\Users\fawke\AppData\Local\Temp\ipykernel_35264\2555325896.py:98: RuntimeWarning: invalid value encountered in power
  return np.where(r >= 0, r ** 0.88, -2.25 * (-r) ** 0.88)
C:\Users\fawke\AppData\Local\Temp\ipykernel_35264\2555325896.py:98: RuntimeWarning: invalid value encountered in power
  return np.where(r >= 0, r ** 0.88, -2.25 * (-r) ** 0.88)
C:\Users\fawke\AppData\Local\Temp\ipykernel_35264\2555325896.py:98: RuntimeWarning: invalid value encountered in power
  return np.where(r >= 0, r ** 0.88, -2.25 * (-r) ** 0.88)
C:\Users\fawke\AppData\Local\Temp\ipykernel_35264\2555325896.py:98: RuntimeWarning: invalid value encountered 

,Total Return,MDD,Sharpe Ratio,symbol,strategy,year
0,0.006414,0.001751,2.696581,BTC/USDT:USDT,basic_momentum_strategy,2019
1,0.023869,0.006981,1.698848,BTC/USDT:USDT,basic_momentum_strategy,2020
2,0.007432,0.002727,6.380064,BTC/USDT:USDT,basic_momentum_strategy,2021
3,-0.001845,0.003790,-0.622739,BTC/USDT:USDT,ts_strategy,2019
4,-0.010955,0.015550,-1.221103,BTC/USDT:USDT,ts_strategy,2020
...,...,...,...,...,...,...
695,-0.004936,0.014397,-0.397841,CRV/USDT:USDT,maxing_out_strategy,2021
696,-0.003534,0.000000,-4.402796,CRV/USDT:USDT,maxing_out_strategy,2022
697,0.039642,0.004758,3.560312,CRV/USDT:USDT,salience_strategy,2020
698,0.000188,0.010384,0.016189,CRV/USDT:USDT,salience_strategy,2021


In [139]:
# Group by strategy and year
grouped_performance = performance_df.groupby(['year', 'strategy']).agg({
    'Total Return': 'mean',
    'MDD': 'mean',
    'Sharpe Ratio': 'mean'
}).reset_index()

# 결과 출력
grouped_performance[grouped_performance['year']!=2019]

,year,strategy,Total Return,MDD,Sharpe Ratio
7,2020,52w_high_strategy,-0.005308,0.011800,-0.836224
8,2020,basic_momentum_strategy,0.017984,0.004314,1.986064
9,2020,capital_gains_strategy,0.003765,0.001655,0.941582
10,2020,maxing_out_strategy,-0.000037,0.006131,-0.088471
11,2020,salience_strategy,0.001031,0.011034,0.079172
12,2020,tk_strategy,0.000823,0.004460,0.044110
13,2020,ts_strategy,-0.004058,0.009535,-0.953746
14,2021,52w_high_strategy,-0.002978,0.021021,-0.413477
15,2021,basic_momentum_strategy,0.066054,0.006870,3.025232
16,2021,capital_gains_strategy,0.016971,0.012336,0.783887


In [ ]:
'''
결론 1 : 기본 모멘텀이 최고다
결론 2 : 다른거 더럽게 안맞는다. 기간이 짧기도 하고 구현에 한계가 있는 문제도 있다.
'''